<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/butterfly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.2 MB/s eta 0:00:00


In [2]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [3]:
SQL_TEMPLATE = """
WITH t as (
SELECT
    query,
    platform,
    count(*) AS cnt,
    sum(count(*)::int) OVER (PARTITION BY query) as query_total,
    sum(count(*)::int) OVER (PARTITION BY platform) as platform_total
FROM
    vizro.yandex_data yd
WHERE
    ts BETWEEN '{start_date}' AND '{end_date}'
GROUP BY
    query, platform)

SELECT
    query,
    sum(case when platform = 'desktop' then cnt else 0 end)::int as count_desktop,
    sum(case when platform = 'touch' then cnt else 0 end)::int as count_touch,
    max(case when platform = 'desktop' then platform_total else 0 end) as desktop_total,
    max(case when platform = 'touch' then platform_total else 0 end) as touch_total
FROM t
    WHERE t.query_total >= {min_cnt}
GROUP BY query
"""

In [4]:
def get_table_data(sql=SQL_TEMPLATE, min_cnt=50, range=['2021-09-08', '2021-09-21']) -> pd.DataFrame:
  start_date, end_date = range
  sql = SQL_TEMPLATE.format(start_date=start_date, end_date=end_date, min_cnt=min_cnt)
  query_df = select(sql)
  #query_df = pd.concat([query_df.sort_values(by="count_desktop", ascending=False).head(10),  query_df.sort_values(by="count_touch", ascending=False).head(10)], axis=0).drop_duplicates(subset=['query'])
  query_df['pct_desktop'] = query_df['count_desktop'] / query_df['desktop_total']
  query_df['pct_touch'] = query_df['count_touch'] / query_df['touch_total']
  query_df["pval"] = query_df.apply(proportions_chi2, axis=1)
  return query_df

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import vizro.models as vm
from vizro import Vizro
from vizro.models.types import capture
from vizro.managers import data_manager


data_manager['table_data'] = get_table_data

@capture("graph")
def butterfly(data_frame: pd.DataFrame, **kwargs) -> go.Figure:
    fig = px.bar(data_frame, **kwargs)

    orientation = fig.data[0].orientation
    x_or_y = "x" if orientation == "h" else "y"

    fig.update_traces({f"{x_or_y}axis": f"{x_or_y}2"}, selector=1)
    fig.update_layout({f"{x_or_y}axis2": fig.layout[f"{x_or_y}axis"]})
    fig.update_layout(
        {
            f"{x_or_y}axis": {"autorange": "reversed", "domain": [0, 0.5]},
            f"{x_or_y}axis2": {"domain": [0.5, 1]},
        }
    )

    if orientation == "h":
        fig.add_vline(x=0, line_width=2, line_color="grey")
    else:
        fig.add_hline(y=0, line_width=2, line_color="grey")
    fig.data[0].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[0]}<extra></extra>'
    fig.data[0].name = 'desktop'
    fig.data[1].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[1]}<extra></extra>'
    fig.data[1].name = 'touch'
    fig.update_yaxes(categoryorder='min ascending')

    return fig



fig = butterfly(
    'table_data',
    x=["pct_desktop", "pct_touch"],
    y="query",
    labels={"value": "% of all", "variable": "platform:"},
    hover_name="query", hover_data={'query': False, 'count_desktop': True, 'count_touch': True },
)

page = vm.Page(title="My page", components=[vm.Graph(figure=fig)])
dashboard = vm.Dashboard(pages=[page])
Vizro().build(dashboard).run()

<IPython.core.display.Javascript object>